In [152]:
import numpy as np
from matplotlib.patches import Ellipse
from skimage.feature import peak_local_max
from skimage.measure import label, regionprops
from sklearn import mixture
import sys
from skimage.draw import circle
import dognet


import matplotlib
import matplotlib.pyplot as plt
import os
from oiffile import imread
from PIL import Image
import random
import skimage
from skimage.transform import resize
import glob
import torch
import sys
sys.path.insert(0, '..')
import scipy.io as sio
import matplotlib.patches as patches
from matplotlib_scalebar.scalebar import ScaleBar
from skimage.transform import rescale
from sklearn.metrics import roc_curve, auc



In [153]:
from skimage.draw import circle
def make_labels(ys,xs,radius=4):
    labels = np.zeros((230,230))
    for xv,yv in zip(xs,ys):
        rr,cc = circle(xv,yv,radius,labels.shape)
        
        labels[rr,cc]=1
    return labels

#Train set
def make_training_set(labels,indexes):
    train_labels = []
    for i in indexes:
        if len(labels[i])==0:
            d = np.zeros((230,230))
        else:
            d = make_labels(np.array(labels[i])[:,0],np.array(labels[i])[:,1])
        train_labels.append(d)
    return train_labels

In [154]:
lines = []
with open("imagesUsed.txt") as fin:
    lines = fin.readlines()
lines.sort()   
print(lines)

['Track0011_01\n', 'Track0011_09\n', 'Track0011_14\n', 'Track0030_00\n', 'Track0030_10\n', 'Track0030_13\n', 'Track0038_01\n', 'Track0040_03\n', 'Track0040_05\n', 'Track0040_14\n', 'Track0045_06\n', 'Track0045_10\n', 'Track0045_14\n', 'Track0050_01\n', 'Track0050_03\n', 'Track0050_07\n', 'Track0063_02\n', 'Track0063_12\n', 'Track0063_15\n', 'Track0070_04\n', 'Track0070_08\n', 'Track0070_14\n', 'Track0080_04\n', 'Track0080_10\n', 'Track0080_13\n', 'Track0084_06\n', 'Track0084_10\n', 'Track0084_14\n', 'Track0090_03\n', 'Track0090_08\n', 'Track0090_11\n', 'Track0092_01\n', 'Track0092_08\n', 'Track0092_15\n', 'Track0110_03\n', 'Track0110_08\n', 'Track0110_15\n', 'Track0113_05\n', 'Track0120_02\n', 'Track0120_06\n', 'Track0120_09\n', 'Track0122_00\n', 'Track0122_09\n', 'Track0122_13\n', 'Track0201_00\n', 'Track0201_04\n', 'Track0201_14\n']


In [151]:
for k in range(0,47):
    name = lines[k]
    name = name[:12]
    
    coordName = "rawCoordinates/%s_rawCoords.npy" % (name)
    with open(coordName) as fin:
        coors = fin.readlines()
    
    pt = np.zeros((len(coors), 2))
    for i in range(0,len(coors)):
        xy = coors[i]
        x=xy[:25]
        y=xy[25:]
        x=float(x)
        y=float(y)
        point=np.arange(2)
        pt[i][0] = x
        pt[i][1] = y

    b = [[] for i in range(0,4)]
    
    for j in pt:
        if j[0] < 460 and j[1] < 460:
            point = np.copy(j)
            (b[0]).append(.5*np.array(point))

        if j[0] > 340 and j[1] < 460:
            point = np.copy(j)
            point[0]=point[0]-340
            (b[1]).append(.5*np.array(point))
      
        if j[0] < 460 and j[1] > 340:
            point = np.copy(j)
            point[1]=point[1]-340
            (b[2]).append(.5*np.array(point))
        
        if j[0] > 340 and j[1] > 340:
            point = np.copy(j)
            point[0]=point[0]-340
            point[1]=point[1]-340
            (b[3]).append(.5*np.array(point))

    train_labels = make_training_set(b,range(0,4))

    smallPicturesNames=['UL','UR','LL','LR']
    
    for j in range(0,4): 
        maskStr="newTrainingMasks/%s_%s_mask.npy" % (name,smallPicturesNames[j])

        #np.save(maskStr, train_labels[j])